# AI部署

## 容器部署

- 深度学习的部署方式，主要是利用 tf serving的方式或者使用onnx的方式，实现了跨平台的模型服务的共享，可以查看onnx的官网： https://onnx.ai/。这方法可以结合方法3进行部署。
- spark模型的部署模式，主流的方法有2种，第一种是PMML文件部署，第二种是使用mleap组件，详情请查看：https://github.com/combust/mleap。第二种方式是一种可以移植的模式，具有一定的普适性，后续可能会有大的应用前景
- 基于docker+k8s+kubeflow+seldon的一体化部署模式，基于容器编排的方式可以快速实现服务的启动和对外输出，这个方式部署方便，当前对于问题排查和定位比较麻烦，需要做很多的额外的开发工作。

Docker-Cheet-sheet - wpppj的文章 - 知乎
https://zhuanlan.zhihu.com/p/50920327


整个模型工程化部署，主要会聚焦几种场景，第一种是跑批任务，第二种是实时服务。

针对跑批任务，我们尝试采用分而治之的优化，采用数据分割，并行预测，再合并预测结果的方式，提高整体的跑批的速度；有个场景，需要跑5000万的预测数据，没有优化之前需要跑3个小时左右，优化后只要大概32分钟就跑完了，整体的效果提升非常明显。此外，还会尝试另外一种方式，把模型封装成UDF，然后发布在hadoop集群上进行跑批，也实现了分布式跑批的场景，提高了效率。

针对实时在线的服务，核心指标主要是QPS和调用时长，我们在利用PMML部署的模型服务，使用简单的模型，比如LR，整体可以实现QPS 1500，95分位的调用时长在15ms以内。但是对于复杂的模型，比如xgb，FM等复杂模型，还有很大的优化空间，所以工程化落地会制约整个算法迭代的速度及其效果。如果有一个好的工程化团队或者模型serving的工具，是能够大幅度提升整体的模型产能的。工程化本身不是最大的挑战性，主要是需要那些具有开发经验的又懂分布式的人才，其实是可以很好的解决这个问题的。

对于模型类的工程化，到底有哪些地方是不一样的。模型结果本质是一堆参数集合，就像一个配置文件，配置文件的格式不同，有类似pth，pkl，xml，pmml等格式的，模型有数据预处理模块，综上来看，模型部署和正常的应用系统开发不同点在模型服务是一个数据流处理过程，模型文件多了一步的解析工作，导致整个工程化的复杂度增加，除了这点之外，其他地方都是一样的。模型服务本身也需要考虑并发、限流、熔断、降级、分流等标准化的工作，从而来提高整体服务的可用性。

[1]: https://zhuanlan.zhihu.com/p/77636947